In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import cobra, cameo 

ImportError: No module named cameo

In [2]:
model=cameo.models.bigg.e_coli_core

In [5]:
#as is shown a model are subclasses inside classes
print type(model)
print type(model.metabolites.ac_c)
print type(model.reactions.ACALD)

<class 'cobra.core.model.Model'>
<class 'cobra.core.metabolite.Metabolite'>
<class 'cobra.core.reaction.Reaction'>


## Building a model!

### Let's suppose we don't have a model backbone

In [58]:
#We need a a new model.
from cobra import Model
new_model=Model()

In [59]:
new_model#model done

Name,None
Memory address,0x07f82649d3d90
Number of metabolites,0
Number of reactions,0
Objective expression,0
Compartments,


In [60]:
#now we need rxn's in to build the network
from cobra import Reaction
alchemy= Reaction('alchemy',name='Pb to Au') #done

In [61]:
alchemy

Reaction identifier,alchemy
Name,Pb to Au
Memory address,0x07f8264974d90
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [62]:
alchemy.build_reaction_string() #we only have a reaction but nothing happends

' --> '

In [63]:
#now we need metabolites to react
from cobra import Metabolite
Au_e=Metabolite('Au_e',name='Gold',formula='Au',compartment='extra_organism')
Pb_e=Metabolite('Pb_e',name='Plumb',formula='Pb',compartment='extra_organism')

In [64]:
Au_e.formula_weight

196.96655

In [65]:
#adding rxn
alchemy.add_metabolites({Pb_e:-1,Au_e:1})

In [66]:
alchemy #raction done but who is doing it?

Reaction identifier,alchemy
Name,Pb to Au
Memory address,0x07f8264974d90
Stoichiometry,Pb_e --> Au_e Plumb --> Gold
GPR,
Lower bound,0.0
Upper bound,1000.0


In [67]:
alchemy.gene_reaction_rule='philosophers_stone'

In [68]:
alchemy

Reaction identifier,alchemy
Name,Pb to Au
Memory address,0x07f8264974d90
Stoichiometry,Pb_e --> Au_e Plumb --> Gold
GPR,philosophers_stone
Lower bound,0.0
Upper bound,1000.0


In [69]:
#add rxn to the model
new_model.add_reaction(alchemy)

In [70]:
new_model

Name,None
Memory address,0x07f82649d3d90
Number of metabolites,2
Number of reactions,1
Objective expression,0
Compartments,extra_organism


In [71]:
new_model.objective=new_model.reactions.alchemy

In [72]:
new_model.optimize() #we have the uptake bound closet

,fluxes,reduced_costs
alchemy,0.0,0.0


In [73]:
new_model.reactions.alchemy.lower_bound=-10
new_model.summary()

IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------
                           alchemy  0


In [74]:
#we need to buil EX_rxn to leave the system
Ex_au_e=Reaction('Ex_au_e',name='gold exchange')
Ex_au_e.add_metabolites({Au_e:-1})

Ex_pb_e=Reaction('Ex_Pb_e',name='plumb exchange')
Ex_pb_e.add_metabolites({Pb_e:-1})
Ex_pb_e.lower_bound=-10
new_model.add_reactions([Ex_au_e,Ex_pb_e])

In [75]:
new_model

Name,None
Memory address,0x07f82649d3d90
Number of metabolites,2
Number of reactions,3
Objective expression,1.0*alchemy - 1.0*alchemy_reverse_a6c67
Compartments,extra_organism


In [78]:
new_model.objective=new_model.reactions.Ex_au_e
new_model.summary()

IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------
Pb_e  10     Au_e  10      Ex_au_e  10


## We have done our first model with a fake (or not) rxn.

## Now let's try a differente method to add reaction to a model

In [79]:
#let's delete the core rxn
new_model.reactions.alchemy.delete()

### This is a method that build reactions from strings

In [80]:
ALCHEMY=Reaction('ALCHEMY',name='ALCHEMY')
new_model.add_reaction(ALCHEMY)
ALCHEMY.build_reaction_from_string('Pb_e --> Au_e',term_split='+')

In [81]:
new_model.optimize()

,fluxes,reduced_costs
Ex_au_e,10.0,0.0
Ex_Pb_e,-10.0,-2.0
ALCHEMY,10.0,0.0


<br>
<br>
<br>
<br>
<br>
## We have set the basis to build a de novo model, but sometime we have a close related organism with a validated model.
## In this case, we can use tools to build a backbone from zero using the genome of the desired organism or an orthology gene list.


### Carvem is a pyhton module that use a genome file and copare the orthology and phylogeny between BiGG DB using Diamond pipeline. 
### It performs gap filling and many quality improving agorithms.

<br>
<br>
![](carveme.png)
<br>
### Genome-scale metabolic model reconstruction with CarveMe
#### CarveMe is a python-based tool for genome-scale metabolic model reconstruction.

### Documentation
### For more details please check: http://carveme.readthedocs.io/
### DOI: https://doi.org/10.1093/nar/gky537




In [1]:
import cameo

In [6]:
universe=cameo.models.universal.metanetx_universal_model_bigg

In [7]:
universe

Name,metanetx_universal_model_bigg
Memory address,0x07f36bfab1190
Number of metabolites,2819
Number of reactions,6416
Objective expression,0
Compartments,


In [4]:
#short alternative module

import cobra

x_model=cobra.Model() #generate a new model

universe=cobra.io.load_json_model('universal_model.json')

def build_model(rxn_list): # introduce the name of the file containing the reactions that mus be in the same folder or indicate the path of the file
    for item in rxn_list:# loop to add reactions one by one
        x_model.add_reaction(universe.reactions.get_by_id(item).copy())
        print item #printe the added reactions
    return x_model



In [5]:
build_model(['EX_glc__D_e','PEPC'])

EX_glc__D_e
PEPC


Name,None
Memory address,0x07f24a2d1b590
Number of metabolites,5
Number of reactions,2
Objective expression,0
Compartments,


In [2]:
import cobra